In [1]:
import posixpath
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime

In [2]:
data_root = "D:/OneDrive/Documents/Cours/4A/SFE/data/"

npi_fronts = gpd.read_file(posixpath.join(data_root, "NPI_fronts/NPI_fronts.shp"), columns=["front_id", "geometry"]).to_crs(epsg=25833).dropna()
# fix type
npi_fronts.columns = ["front_id_float", "geometry"]
npi_fronts.loc[:, "front_id"] = npi_fronts.front_id_float.astype(int)

npi_fronts.loc[:, "mission"] = "NPI 1936-1938 aerial survey"
npi_fronts.loc[:, "timestamp"] = datetime.datetime(1936, 1, 1)

npi_fronts= npi_fronts.loc[:, ["mission", "timestamp", "front_id", "geometry"]]
npi_fronts.head()

,mission,timestamp,front_id,geometry
0,NPI 1936-1938 aerial survey,1936-01-01,77,"LINESTRING (532543.309 8544051.053, 532636.725..."
1,NPI 1936-1938 aerial survey,1936-01-01,161,"LINESTRING (688874.291 8815732.222, 688441.142..."
2,NPI 1936-1938 aerial survey,1936-01-01,176,"LINESTRING (626236.343 8902937.128, 626351.396..."
3,NPI 1936-1938 aerial survey,1936-01-01,177,"LINESTRING (610107.299 8911853.037, 610139.55 ..."
4,NPI 1936-1938 aerial survey,1936-01-01,144,"LINESTRING (581931.237 8893586.913, 581984.338..."


In [3]:
kh_fronts = gpd.read_file(posixpath.join(data_root, "KH-5_ARGON_fronts/KH-5_ARGON_fronts.shp")).to_crs(epsg=25833)
landsat_fronts = gpd.read_file(posixpath.join(data_root, "Landsat_MSS_fronts/Landsat_MSS_fronts.shp")).to_crs(epsg=25833)
all_fronts = pd.concat([kh_fronts, landsat_fronts, npi_fronts], ignore_index=True)
all_fronts.head()

,image,mission,timestamp,RGI60ID,RGI70ID,front_id,front_dir,geometry
0,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00777,RGI2000-v7.0-G-07-01225,3,E,"LINESTRING (447461.671 8832029.666, 447497.042..."
1,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00775,RGI2000-v7.0-G-07-01229,4,SE,"LINESTRING (446131.469 8835920.477, 446232.529..."
2,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00709,RGI2000-v7.0-G-07-00011,25,W,"LINESTRING (416080.008 8819152.093, 416146.96 ..."
3,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00664,RGI2000-v7.0-G-07-00015,26,W,"LINESTRING (415736.404 8813938.659, 415733.877..."
4,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00663,RGI2000-v7.0-G-07-00016,27,W,"LINESTRING (417320.52 8806789.925, 417358.417 ..."


In [4]:
link = pd.DataFrame(
    [],
    columns = ["front_id", "front_dir", "RGI60ID", "RGI70ID"]
)

link.loc[:, "front_id"] = all_fronts.front_id.unique()

link.loc[:, "front_dir"] = link.front_id.apply(lambda i: all_fronts.loc[all_fronts.front_id == i, "front_dir"].values[0])
link.loc[:, "RGI60ID"] = link.front_id.apply(lambda i: all_fronts.loc[all_fronts.front_id == i, "RGI60ID"].values[0])
link.loc[:, "RGI70ID"] = link.front_id.apply(lambda i: all_fronts.loc[all_fronts.front_id == i, "RGI70ID"].values[0])
link.head()

,front_id,front_dir,RGI60ID,RGI70ID
0,3,E,RGI60-07.00777,RGI2000-v7.0-G-07-01225
1,4,SE,RGI60-07.00775,RGI2000-v7.0-G-07-01229
2,25,W,RGI60-07.00709,RGI2000-v7.0-G-07-00011
3,26,W,RGI60-07.00664,RGI2000-v7.0-G-07-00015
4,27,W,RGI60-07.00663,RGI2000-v7.0-G-07-00016


In [6]:
npi_fronts.loc[:, "front_dir"] = npi_fronts.front_id.apply(lambda i: link.loc[link.front_id == i].front_dir.values[0])
npi_fronts.loc[:, "RGI60ID"] = npi_fronts.front_id.apply(lambda i: link.loc[link.front_id == i].RGI60ID.values[0])
npi_fronts.loc[:, "RGI70ID"] = npi_fronts.front_id.apply(lambda i: link.loc[link.front_id == i].RGI70ID.values[0])
npi_fronts = npi_fronts.loc[:, ["mission", "timestamp", "RGI60ID", "RGI70ID", "front_id", "front_dir", "geometry"]]
npi_fronts.head()

,mission,timestamp,RGI60ID,RGI70ID,front_id,front_dir,geometry
0,NPI 1936-1938 aerial survey,1936-01-01,RGI60-07.00006,RGI2000-v7.0-G-07-00787,77,NW,"LINESTRING (532543.309 8544051.053, 532636.725..."
1,NPI 1936-1938 aerial survey,1936-01-01,RGI60-07.00025,RGI2000-v7.0-G-07-01384,161,S,"LINESTRING (688874.291 8815732.222, 688441.142..."
2,NPI 1936-1938 aerial survey,1936-01-01,RGI60-07.00035,RGI2000-v7.0-G-07-01365,176,E,"LINESTRING (626236.343 8902937.128, 626351.396..."
3,NPI 1936-1938 aerial survey,1936-01-01,RGI60-07.00036,RGI2000-v7.0-G-07-01363,177,E,"LINESTRING (610107.299 8911853.037, 610139.55 ..."
4,NPI 1936-1938 aerial survey,1936-01-01,RGI60-07.01449,RGI2000-v7.0-G-07-01343,144,NW,"LINESTRING (581931.237 8893586.913, 581984.338..."


In [ ]:
# # /!\ file modification
# npi_fronts.to_file(posixpath.join(data_root, "NPI_fronts/NPI_fronts.shp"), index=False)

C:\Users\loris\AppData\Roaming\Python\Python312\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field timestamp create as date field, though DateTime requested.
  ogr_write(
